# Setup

In [1]:
!pwd

/home/umni2/a/umnilab/users/verma99/mk/home_detection/code


**New `meanshift`**: To speed up the performance of the clustering algorithm, some key redundancies in the existing implementation of `sklearn.cluster.MeanShift` were removed. These include:
* Removing the labeling part which involves assigning cluster labels to all the sample points. This is not necessary for home detection since we only care about the cluster center. This prevents computation of distances between the cluster centers and the sample points using the BallTree algorithm, which is a massive speedup.
* Removing the cluster merging part. Cluster merging involves absorbing smaller clusters in close vicinity (within radius threshold) to larger clusters. Since we only care about the largest cluster and the largest cluster is always included in this step, there is no need to perform this step.

In [2]:
import clustering
from mobilkit.umni import *
from setup import P, Region, Dataset

In [3]:
import contextily as ctx
import haversine as hs
import shapely
from sklearn.cluster import AgglomerativeClustering

In [4]:
SP.start()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/27 16:36:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load datasets and regions

In [5]:
datasets = [Dataset(k, *v.values()) for k, v in P.params.get('datasets').items()]
ds2 = datasets[1]; ds2

Dataset D2(Baton Rouge: 2021-08-26 - 2021-09-07)

In [6]:
regions = sorted(list({x.region for x in datasets}), key=lambda x: x.name)
regions

[Region(Austin), Region(Baton Rouge), Region(Houston), Region(Indianapolis)]

# Get home locations

## $A_1$: Centroid

In [7]:
def get_homes_centroid(ds, save=True, overwrite=False):
    outpath = ds.data / 'homes/A1.parquet'
    if outpath.exists() and not overwrite:
        return SP.read_parquet(outpath)
    print(f'Processing {ds}')
    start_time = dt.datetime.now()
    df = SP.read_parquet(ds.data / 'night_pings')
    df = df.select(UID, *[
        F.udf(lambda x: sum(x) / len(x), T.float)(x).alias(x) 
        for x in [LON, LAT]]).sort(UID)
    if save:
        df = df.toPandas()
        df.to_parquet(U.mkfile(outpath))
    print(f'Runtime: {dt.datetime.now() - start_time}')
    return df

# %time x = get_homes_centroid(ds2); x

In [8]:
# total time for aus + baro + hous + indy + sant = 41s
# t=1:32 {1=>0:16, 2=>0:04, 3=>0:32, 4=>0:07, 5=>0:06, 6=>0:10, 7=>0:05, 8=>0:12}
for ds in datasets:
    get_homes_centroid(ds, overwrite=False)

## $A_2$: Grid frequency-based

In [9]:
grid_cell_size = 20 # meters
P.params.set({'algorithms': {'grid_frequency': {'cell_size': grid_cell_size}}})

In [10]:
def get_homes_grid_freq(ds, cell_size=grid_cell_size, save=True, overwrite=False):
    outfile = ds.data / 'homes/A2.parquet'
    if outfile.exists() and not overwrite:
        return SP.read_parquet(outfile)
    print(f'Processing {ds}')
    start_time = dt.datetime.now()
    bbox = shapely.box(*ds.region.bbox)
    minLon, minLat, maxLon, maxLat = ds.region.bbox
    bbox = Gdf({'geometry': [bbox]}, crs=CRS_DEG)
    minX, minY, maxX, maxY = bbox.to_crs(CRS_M).bounds.iloc[0]
    scaleX = (maxX - minX) / (maxLon - minLon) / cell_size
    scaleY = (maxY - minY) / (maxLat - minLat) / cell_size
    nCols = int(np.ceil((maxX - minX) / cell_size))
    nRows = int(np.ceil((maxY - minY) / cell_size))
    df = SP.read_parquet(ds.data / 'night_pings')
    df = df.groupby(UID).agg(*[F.flatten(F.collect_list(x)).alias(x) for x in [LON, LAT]])
    
    def udf(x, y):
        cx = np.floor((Arr(x) - minLon) * scaleX)
        cy = np.floor((Arr(y) - minLat) * scaleY)
        cells, freq = np.unique(np.vstack([cx, cy]).T, axis=0, return_counts=True)
        cx, cy = cells[freq.argmax()]
        return int(cx), int(cy)
    df = df.select(UID, F.udf(udf, T.array(T.int64))(LON, LAT).alias('cell_id'))
    df = df.select(UID, *[F.col('cell_id')[i].alias(x) for i, x in enumerate(['cx', 'cy'])])
    df = df.withColumn(LON, minLon + (F.col('cx') + 0.5) * ((maxLon - minLon) / nCols))
    df = df.withColumn(LAT, minLat + (F.col('cy') + 0.5) * ((maxLat - minLat) / nRows))
    df = df.select(UID, *[F.col(x).cast(T.float).alias(x) for x in [LON, LAT]])
    if save:
        df = df.toPandas()
        df.to_parquet(U.mkfile(outfile))
    print(f'Runtime: {dt.datetime.now() - start_time}')
    return df

# %time x = get_homes_grid_freq(ds2, overwrite=True, save=False); x

In [11]:
# t=1:51 ({1=>0:11, 2=>0:04, 3=>0:42, 4=>0:08, 5=>0:07, 6=>0:14, 7=>0:07, 8=>0:17})
for ds in datasets:
    get_homes_grid_freq(ds, overwrite=False)

## A3 & A4: Meanshift clustering
In the case of $A_2$, for each slot, the pings are aggregated to just the mean "super-ping".

In [12]:
meanshift_radius = 250 # meters
meanshift_params = dict(bin_seeding=True, min_bin_freq=2, max_iter=50)
superping_slot_size = 30 * 60 # seconds
P.params.set({'algorithms': {'meanshift': {'radius': meanshift_radius} | meanshift_params,
              'superping': {'slot_size': superping_slot_size}}})

In [13]:
def get_homes_meanshift(ds, use_superpings, radius=meanshift_radius, 
                        slot_size=superping_slot_size, 
                        params=meanshift_params, overwrite=False):
    outpath = ds.data / 'homes/{}.parquet'.format(
        'A4' if use_superpings else 'A3')
    if outpath.exists() and not overwrite:
        return SP.read_parquet(outpath)
    print(f'Processing {ds}')
    start_time = dt.datetime.now()
    _, miny, _, maxy = ds.region.bbox
    bandwidth = mk.geo.dist_m2deg(radius, (miny + maxy) / 2)
    
    def udf(x, y):
        model = clustering.MeanShift(bandwidth=bandwidth, **params)
        model.fit(np.vstack([x, y]).T)
        cx, cy = [float(x) for x in model.cluster_center]
        return (cx, cy, float(model.used_mean))
    df = SP.read_parquet(ds.data / 'night_pings')
    if use_superpings:
        df = df.select(UID, F.arrays_zip(LON, LAT, TS).alias('xyt'))
        df = df.select(UID, F.explode('xyt').alias('xyt'))
        df = df.select(UID, *[F.col('xyt')[x].alias(x) for x in [LON, LAT, TS]])
        df = df.withColumn('slot', F.udf(lambda t: int(t / slot_size), T.int16)(TS))
        df = df.groupby(UID, 'slot').agg(
            *[F.mean(x).cast(T.float).alias(x) for x in [LON, LAT]])
        df = df.groupby(UID).agg(*[F.collect_list(x).alias(x) for x in [LON, LAT]])
    df = df.withColumn('home', F.udf(udf, T.array(T.float))(LON, LAT))
    df = df.select(UID, *[F.col('home')[i].alias(x) for i, x 
                          in enumerate([LON, LAT, 'used_mean'])])
    df = df.withColumn('used_mean', F.col('used_mean').cast(T.bool))
    df.toPandas().to_parquet(U.mkfile(outpath))
    print(f'Runtime: {dt.datetime.now() - start_time}')
    return df
    
# %time x = get_homes_meanshift(ds2, overwrite=True); x

### $A_3$: All-time clustering

In [14]:
# t=13:56 (1=>1:11, 2=>0:22, 3=>3:22, 4=>1:21, 5=>1:31, 6=>2:43, 7=>0:51, 8=>2:34)
for ds in datasets:
    get_homes_meanshift(ds, use_superpings=False, overwrite=False)

### $A_4$: Superping clustering

In [15]:
# t=14:50 ({1=>1:28, 2=>0:28, 3=>4:54, 4=>1:11, 5=>1:14, 6=>2:17, 7=>0:50, 8=>2:28})
for ds in datasets:
    get_homes_meanshift(ds, use_superpings=True, overwrite=False)

## A3: Stay point method
From [Sagedhinasr et al. (2019)](https://doi.org/10.1061/9780784482438.002)

### Generate stay points
From [Li et al. (2008)](https://dl.acm.org/doi/10.1145/1463434.1463477)

In [16]:
base_day_hrs = (7, 19) # 7am – 7pm
min_total_nightly_pts = 10

In [17]:
stay_pt_method_params = {
    # distance threshold (meters) for stay point detection
    'dist_thresh': 250,
    # time threshold (in seconds) for stay point detection
    'time_thresh': 30 * 60,
    # maximum intra-cluster distance (meters) for a stay region
    'intra_clust_dist': 250,
    # minimum total dwell time of a stay region (seconds) to 
    # be considered an eligible home location
    'min_total_dwell': 24 * 3600, # seconds
    # minimum dwell time (seconds) during night to be
    # considered eligible an eligible home location
    'min_night_dwell': 3 * 3600, # seconds
}

In [18]:
def get_stay_points(x, y, t, dist_thresh, time_thresh):
    stay_pts = []
    i = 0
    while i < len(x):
        j = i + 1
        while j < len(x):
            dist = float(hs.haversine((y[i], x[i]), (y[j], x[j]), unit='m'))
            if dist > dist_thresh:
                if t[j] - t[i] > time_thresh:
                    stay_pts.append({
                        LON: sum(x[i:j]) / (j - i),
                        LAT: sum(y[i:j]) / (j - i),
                        TS + '_arr': t[i],
                        TS + '_dep': t[j]})
                i = j
                break
            j += 1
        if j == len(x):
            break
    return stay_pts

### Generate stay regions

In [19]:
def get_stay_regions_homes(df, intra_clust_dist, linkage='complete'):
    if isinstance(df, Sdf):
        df = df.toPandas()
    df = mk.geo.pdf2gdf(df, LON, LAT, CRS_DEG).to_crs(CRS_M)
    pts = mk.geo.gdf2pdf(df, LON, LAT, CRS_M).set_index(df[UID])
    model = AgglomerativeClustering(
        distance_threshold=intra_clust_dist, n_clusters=None,
        linkage=linkage, compute_full_tree=True)
    homes = []
    for uid, df in tqdm(pts.groupby(UID)):
        if len(df) == 1:
            homes.append({UID: uid, LON: df[LON].iloc[0], 
                          LAT: df[LAT].iloc[0], 'used_mean': True})
        else:
            model.fit(df.values)
            clusts, freq = np.unique(model.labels_, return_counts=True)
            home_clust = clusts[freq.argmax()]
            xy = df[model.labels_ == home_clust].mean(axis=0)
            homes.append({UID: uid, LON: xy[LON], LAT: xy[LAT], 
                          'used_mean': False})
    homes = mk.geo.pdf2gdf(Pdf(homes), LON, LAT, CRS_M)
    xy = mk.geo.gdf2pdf(homes.to_crs(CRS_DEG), LON, LAT)
    homes = pd.concat([homes[UID], xy, homes['used_mean']], axis=1)
    return homes

### Main

In [20]:
def get_homes_stay_pts(ds, dist_thresh, time_thresh,
                       min_total_dwell, min_night_dwell,
                       intra_clust_dist, day_hrs,
                       save=True, overwrite=False):
    outpath = ds.data / 'homes/A5.parquet'
    if outpath.exists() and not overwrite:
        return SP.read_parquet(outpath)
    print(f'Processing {ds}')
    start_time = dt.datetime.now()
    # collect pings
    df = []
    for date in ds.dates:
        d = SP.read_parquet(ds.data / f'pings/{date}')
        nDays = (date - ds.start).days
        def add_day(t): return [t + nDays * 86400 for t in t]
        d = d.withColumn(TS, F.udf(add_day, T.array(T.float))(TS))
        df.append(d)
    df = reduce(Sdf.union, df)
    df = df.groupby(UID).agg(*[F.flatten(F.collect_list(x)).alias(x) 
                               for x in [LON, LAT, TS]])
    # get stay points
    df = df.select(UID, F.udf(
        lambda x, y, t: get_stay_points(x, y, t, dist_thresh, time_thresh), 
        T.array(T.map(T.str, T.float)))(LON, LAT, TS).alias('stay_pts'))
    df = (df.select(UID, F.explode('stay_pts').alias('stay_pt'))
          .select(UID, *[F.col('stay_pt')[x].alias(x) for x in 
                         [LON, LAT, 'ts_arr', 'ts_dep']]))
    # filter stay points
    df = df.withColumn('dwell_time', F.col('ts_dep') - F.col('ts_arr'))
    df1 = df.filter(f'dwell_time >= {min_total_dwell}')
    morning_start, night_start = [x * 3600 for x in day_hrs]
    df = (df.filter(f'dwell_time < {min_total_dwell}')
          .withColumn('t1', morning_start - (F.col('ts_arr')))
          .withColumn('t2', morning_start - (F.col('ts_dep') % 86400))
          .withColumn('t3', (F.col('ts_arr') % 86400) - night_start)
          .withColumn('t4', (F.col('ts_dep') % 86400) - night_start))
    for t in ['t1','t2','t3','t4']:
        df = df.withColumn(t, F.when(F.col(t) < 0, 0).otherwise(F.col(t)))
    df = df.withColumn('night_dwell', F.expr('t1 + t2 + t3 + t4'))
    df = df.filter(f'night_dwell >= {min_night_dwell}')
    df = df.select(UID, LON, LAT).union(df1.select(UID, LON, LAT))
    # get the stay regions (clustering) and idenitfy the home location
    homes = get_stay_regions_homes(df, intra_clust_dist)
    if save:
        homes.to_parquet(outpath)
    print(f'Runtime: {dt.datetime.now() - start_time}')
    return homes

# %time x = get_homes_stay_pts(ds2, day_hrs=base_day_hrs, save=False, **stay_pt_method_params); x

In [21]:
# t=54:09 ({1=>4:58, 2=>1:47, 3=>,12:54 4=>5:27, 5=>5:28, 6=>10:00, 7=>3:34, 8=>10:01})
for ds in datasets:
    get_homes_stay_pts(ds, overwrite=False, day_hrs=base_day_hrs, **stay_pt_method_params)

# Visualize

## Home locations

In [22]:
algos = ['A1', 'A2', 'A3', 'A4', 'A5']

In [23]:
def plot_home_locs(rgn, homes_var, grid_size=500, vmax=None,
                   size=(12, 12), dpi=120, markersize=20, cmap='Reds'):
    """
    grid_size: float
        Size of the grid cell (meters)
    """
    h = getattr(rgn, 'h' + homes_var).copy()
    h = mk.geo.pdf2gdf(h, LON, LAT, CRS_DEG).to_crs(CRS_M)
    h = pd.concat([h, mk.geo.gdf2pdf(h, 'x', 'y', CRS_M)], axis=1).dropna()
    for x in ['x', 'y']:
        h[x] = (h[x] / grid_size).astype(int) * grid_size
    h = h.groupby(['x', 'y']).size().rename('n_users').reset_index()
    h = mk.geo.pdf2gdf(h, 'x', 'y', CRS_M)
    minx, miny, maxx, maxy = rgn.cnty.to_crs(CRS_M).unary_union.bounds
    ax = U.plot(size=size, dpi=dpi, xeng=1, yeng=1, title=f'Method: {homes_var}',
                # xlim=(minx, maxx), ylim=(miny, maxy),
                xlab='Distance from westernmost edge (km)',
                ylab='Distance from southernnmost edge (km)')
    boundary = rgn.boundary.to_crs(CRS_M)
    h = gpd.sjoin(h, boundary, predicate='within')
    h.plot(ax=ax, column='n_users', cmap=cmap, markersize=markersize,
           legend=True, alpha=0.5, vmax=vmax, zorder=100, marker='s', edgecolor='none',
           legend_kwds=dict(shrink=0.5, label='No. of users'))
    boundary.plot(ax=ax, edgecolor='b', facecolor='none', linewidth=2)
    # rgn.cnty.to_crs(CRS_M).plot(ax=ax, facecolor='none', edgecolor='k',
    #                             linewidth=2, zorder=101)
    ctx.add_basemap(ax=ax, source=ctx.providers.Stamen.TonerLite, crs=CRS_M)

In [24]:
def plot_home_locs_diff(rgn, grid_size=500, vmin=None, vmax=None,
                        size=(12, 12), dpi=120, markersize=20, cmap='RdBu'):
    """
    grid_size: float
        Size of the grid cell (meters)
    """
    hold = getattr(rgn, 'hA1').copy()
    hnew = getattr(rgn, 'hA2').copy()
    hold = mk.geo.pdf2gdf(hold, LON, LAT, CRS_DEG).to_crs(CRS_M)
    hnew = mk.geo.pdf2gdf(hnew, LON, LAT, CRS_DEG).to_crs(CRS_M) 
    hold = pd.concat([hold, mk.geo.gdf2pdf(hold, 'x', 'y', CRS_M)], axis=1).dropna()
    hnew = pd.concat([hnew, mk.geo.gdf2pdf(hnew, 'x', 'y', CRS_M)], axis=1).dropna()
    for x in ['x', 'y']:
        hold[x] = (hold[x] / grid_size).astype(int) * grid_size
        hnew[x] = (hnew[x] / grid_size).astype(int) * grid_size
    hold = hold.groupby(['x', 'y']).size().rename('n_old').fillna(0)
    hnew = hnew.groupby(['x', 'y']).size().rename('n_new').fillna(0)
    h = pd.concat([hold, hnew], axis=1).fillna(0).reset_index()
    h['delta'] = h['n_new'] - h['n_old']
    h = mk.geo.pdf2gdf(h, 'x', 'y', CRS_M)
    minx, miny, maxx, maxy = rgn.cnty.to_crs(CRS_M).unary_union.bounds
    ax = U.plot(size=size, dpi=dpi, xeng=1, yeng=1,
                # xlim=(minx, maxx), ylim=(miny, maxy),
                xlab='Distance from westernmost edge (km)',
                ylab='Distance from southernnmost edge (km)')
    boundary = rgn.boundary.to_crs(CRS_M)
    h = gpd.sjoin(h, boundary, predicate='within')
    h.plot(ax=ax, column='delta', cmap=cmap, markersize=markersize,
           legend=True, alpha=0.5, vmin=vmin, vmax=vmax, zorder=100, marker='s', edgecolor='none',
           legend_kwds=dict(shrink=0.5, label='No. of users'))
    boundary.plot(ax=ax, edgecolor='b', facecolor='none', linewidth=2)
    ctx.add_basemap(ax=ax, source=ctx.providers.Stamen.TonerLite, crs=CRS_M)    

In [25]:
def plot_representativeness_cbg(rgn, homes_var, vmin=0, vmax=15,
                                size=(10, 10), dpi=120, cmap='magma_r'):
    homes = getattr(rgn, 'h' + homes_var).copy()
    homes = mk.geo.pdf2gdf(homes, LON, LAT, CRS_DEG)
    acs = gpd.read_file(rgn.data / 'geometry/acs.gpkg').query('popu > 0')
    homes = gpd.sjoin(homes, acs[['geometry', 'geoid']], predicate='within')
    n_users = homes.groupby('geoid').size().rename('n_users').reset_index()
    acs = acs.merge(n_users, on='geoid')[['county', 'n_users', 'popu', 'geometry']]
    acs['repres'] = (acs['n_users'] * 100) / acs['popu']
    ax = U.plot(size=size, dpi=dpi, axoff=1)
    acs = gpd.sjoin(acs, rgn.boundary)
    acs.to_crs(CRS_M).plot(
        ax=ax, column='repres', cmap=cmap, legend=True, 
        vmin=vmin, vmax=vmax, edgecolor='none', # classification_kwds=dict(bins=np.arange(vmin, vmax+1)),
        legend_kwds=dict(shrink=0.5, label='Data representativeness (%)'))
    counties = acs.groupby('county')['geometry'].agg(lambda x: x.unary_union).set_crs(CRS_DEG)
    counties.to_crs(CRS_M).plot(ax=ax, facecolor='none', edgecolor='k', linewidth=2)
    rgn.boundary.to_crs(CRS_M).plot(ax=ax, facecolor='none', edgecolor='b', linewidth=2)

# plot_representativeness_cbg(regions[-1], 'A4')